In [1]:
import pyspark as ps
from pyspark.sql.functions import col
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("Colorado-Addresess") \
            .getOrCreate()
sc = spark.sparkContext

In [2]:
# df = spark.read.format("csv").option("header", "true").load("urbanAddresses.csv")

In [2]:
df = spark.read.csv('urbanAddresses.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)

In [5]:
# df.printSchema()

root
 |-- ADDRESS_TYPE: string (nullable = true)
 |-- X_COORD: double (nullable = true)
 |-- Y_COORD: double (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ADDRESS_NUMBER_PREFIX: string (nullable = true)
 |-- ADDRESS_NUMBER: double (nullable = true)
 |-- ADDRESS_NUMBER_SUFFIX: string (nullable = true)
 |-- PREMODIFIER: string (nullable = true)
 |-- PREDIRECTIONAL: string (nullable = true)
 |-- STREET_NAME: string (nullable = true)
 |-- POSTTYPE: string (nullable = true)
 |-- POSTDIRECTIONAL: string (nullable = true)
 |-- POSTMODIFIER: string (nullable = true)
 |-- BUILDING_TYPE: string (nullable = true)
 |-- BUILDING_IDENTIFIER: string (nullable = true)
 |-- UNIT_TYPE: string (nullable = true)
 |-- UNIT_IDENTIFIER: string (nullable = true)
 |-- COMPOSITE_UNIT_TYPE: string (nullable = true)
 |-- COMPOSIT_UNIT_IDENTIFIER: string (nullable = true)
 |-- FULL_ADDRESS: string (nullable = true)



In [6]:
# df.show()
# df.count()

325086

In [3]:
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType

# broomfield_df.select('FULL_ADDRESS').filter(col('CITY') == 'BROOMFIELD').show()

# rename columns
df = df.withColumnRenamed("FULL_ADDRESS", "address")

df_sample = df.select('address').distinct().rdd.takeSample(False, 1000, seed=463)

# Create a schema for the dataframe
schema = StructType([
    StructField('address', StringType(), True),
])

# create new dataframe yoooo
new_df = spark.createDataFrame(df_sample, schema)

# df.select('STREET_NAME').rdd.takeSample(False, 250, seed=0)


In [4]:
# new_df.show()
from pyspark.sql.functions import lit


new_df = new_df.withColumn("city", lit('Denver'))\
               .withColumn('state', lit('CO'))

# new_df.show(250, truncate=False)

# new_df.select('address').filter()

# count = new_df.count()
# for i in range(count):
#     printnew_df.collect()[i]['address'])

# print(new_df.collect()[0]['city'] + ' '+ new_df.collect()[0]['state'])
test_df = new_df.toPandas()
test_df.head()

# for i in range(5):
#     print(test_df.loc[i,'address'])

test_df = test_df.apply(lambda x: x.astype(str).str.lower())
test_df['address'] = test_df['address'].apply(lambda x: x.split())
test_df.head()
# new_df = lower_address(new_df)
# df.withColumn('age2', df.age + 2).collect()

,address,city,state
0,"[1430, larimer, st]",denver,co
1,"[2555, 17th, st, unit, 203]",denver,co
2,"[153, n, vallejo, st, unit, 103]",denver,co
3,"[19011, e, 57th, pl]",denver,co
4,"[652, n, inca, st]",denver,co


In [5]:
def clean_wrds(df, col='address'):
    
    bad_wrds = ['bldg', 'unit', 'apt', '#', 'irrp']
    clear_wrds = df.loc[:, col]
    for i, lst in enumerate(clear_wrds,0):
        for wrd in lst:
            if wrd in bad_wrds:
                lst.remove(wrd)
                lst.pop(-1)
        lst = ' '.join(lst)
        clear_wrds[i] = lst

    return clear_wrds

clean_wrds(test_df)
test_df.head()


,address,city,state
0,1430 larimer st,denver,co
1,2555 17th st,denver,co
2,153 n vallejo st,denver,co
3,19011 e 57th pl,denver,co
4,652 n inca st,denver,co


In [7]:
test_df.head()

,address,city,state
0,1430 larimer st,denver,co
1,2555 17th st,denver,co
2,153 n vallejo st,denver,co
3,19011 e 57th pl,denver,co
4,652 n inca st,denver,co
